In [42]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 2

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[:2000].reshape(2000, 784)
x_test = x_test[2000:3000].reshape(1000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train[:2000], num_classes)
y_test = keras.utils.to_categorical(y_test[2000:3000], num_classes)

2000 train samples
1000 test samples


In [68]:
def network(x_train, y_train, x_test, y_test, d_1, d_2):
    model = Sequential()
    model.add(Dense(d_1, activation='relu', input_shape=(784,)))
    model.add(Dense(d_2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return float(score[0])    #this float is important

In [69]:
from nevergrad import instrumentation as instru

def myfunction(arg1, arg2, arg3, value=3):
    print(arg1, arg2, arg3)
    return value**2

# argument transformation
first_dense_neurons = instru.variables.OrderedDiscrete([10, 20])  # neurons of the first dense layer
second_dense_neurons = instru.variables.OrderedDiscrete([10, 30])  # 1st arg. = positional discrete argument
dropout_rate = instru.variables.OrderedDiscrete([0, 0.2, 0.6])  # 2nd arg. = positional discrete argument

# create the instrumented function
ifunc = instru.InstrumentedFunction(network, x_train, y_train, x_test, y_test,
                                    first_dense_neurons, second_dense_neurons)
# the 3rd arg. is a positional arg. which will be kept constant to "blublu"
print(ifunc.dimension)  # 5 dimensional space

# The dimension is 5 because:
# - the 1st discrete variable has 1 possible values, represented by a hard thresholding in a 1-dimensional space, i.e. we add 1 coordinate to the continuous problem
# - the 2nd discrete variable has 3 possible values, represented by softmax, i.e. we add 3 coordinates to the continuous problem
# - the 3rd variable has no uncertainty, so it does not introduce any coordinate in the continuous problem
# - the 4th variable is a real number, represented by single coordinate.

ifunc([1, -80])  # will print "b e blublu" and return 49 = (mean + std * arg)**2 = (1 + 2 * 3)**2
# b is selected because 1 > 0 (the threshold is 0 here since there are 2 values.
# e is selected because proba(e) = exp(80) / (exp(80) + exp(-80) + exp(-80))







2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_283 (Dense)            (None, 20)                15700     
_________________________________________________________________
dense_284 (Dense)            (None, 10)                210       
_________________________________________________________________
dense_285 (Dense)            (None, 10)                110       
Total params: 16,020
Trainable params: 16,020
Non-trainable params: 0
_________________________________________________________________
Train on 2000 samples, validate on 1000 samples
Epoch 1/2
2000/2000 [==============================] - 4s 2ms/step - loss: 2.2029 - acc: 0.2245 - val_loss: 2.0891 - val_acc: 0.2570
Epoch 2/2
2000/2000 [==============================] - 0s 53us/step - loss: 1.9658 - acc: 0.3055 - val_loss: 1.9094 - val_acc: 0.3000
Test loss: 1.9094392070770263
Test accuracy: 0.3


1.9094392070770263

In [70]:
'''
not ask and tell


from nevergrad.optimization import optimizerlib

optimizer = optimizerlib.CMA(dimension=2, budget=5, num_workers=1)

from concurrent import futures

with futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:
    recommendation = optimizer.optimize(ifunc, executor=executor, batch_mode=True)
'''

'\nnot ask and tell\n\n\nfrom nevergrad.optimization import optimizerlib\n\noptimizer = optimizerlib.CMA(dimension=2, budget=5, num_workers=1)\n\nfrom concurrent import futures\n\nwith futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:\n    recommendation = optimizer.optimize(ifunc, executor=executor, batch_mode=True)\n'

In [71]:
for _ in range(optimizer.budget):
    print('epoch: ', _)
    x = optimizer.ask()
    print('X: ', x)
    value = ifunc(x)
    print('value: ', value)
    optimizer.tell(x, value)
    print('optimizer: ', optimizer)

recommendation = optimizer.provide_recommendation()

epoch:  0
X:  [ 0.62114414 -0.8178619 ]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_286 (Dense)            (None, 20)                15700     
_________________________________________________________________
dense_287 (Dense)            (None, 10)                210       
_________________________________________________________________
dense_288 (Dense)            (None, 10)                110       
Total params: 16,020
Trainable params: 16,020
Non-trainable params: 0
_________________________________________________________________
Train on 2000 samples, validate on 1000 samples
Epoch 1/2
2000/2000 [==============================] - 4s 2ms/step - loss: 2.0553 - acc: 0.2775 - val_loss: 1.9029 - val_acc: 0.4040
Epoch 2/2
2000/2000 [==============================] - 0s 53us/step - loss: 1.6213 - acc: 0.5675 - val_loss: 1.6091 - val_acc: 0.5480
Test loss: 1.6091353273391724
Test accuracy: 0.

In [72]:
type(recommendation)

tuple

In [73]:
print(recommendation)

(0.7408146354361135, 0.5331237899686446)


In [74]:
ifunc(recommendation)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_301 (Dense)            (None, 20)                15700     
_________________________________________________________________
dense_302 (Dense)            (None, 30)                630       
_________________________________________________________________
dense_303 (Dense)            (None, 10)                310       
Total params: 16,640
Trainable params: 16,640
Non-trainable params: 0
_________________________________________________________________
Train on 2000 samples, validate on 1000 samples
Epoch 1/2
2000/2000 [==============================] - 5s 2ms/step - loss: 2.0320 - acc: 0.3235 - val_loss: 1.8295 - val_acc: 0.4320
Epoch 2/2
2000/2000 [==============================] - 0s 53us/step - loss: 1.5732 - acc: 0.6080 - val_loss: 1.4507 - val_acc: 0.6380
Test loss: 1.4506826238632202
Test accuracy: 0.638


1.4506826238632202